## 前期准备

alice 和 bob 节点分别启动 ray 集群。

```
ray start --head --node-ip-address="ip" --port="port" --include-dashboard=False --disable-usage-stats
```

屏幕输出中显示 <span style="color: #008A00">Ray runtime started</span>，则说明 Ray 的主节点启动成功。

## 初始化 SecretFlow

alice 和 bob 节点都需要初始化 secretflow，初始化语句只有在 self_party 上有区别。

1. 分别使用 Ray 主节点的 node-ip-address 和 port 填充 sf.init 的 address 参数。
2. parties 中 alice 和 bob 的 address 分别填可以被对方访问的地址，并且选择一个**未被占用的端口。**

In [1]:
import secretflow as sf
import spu

network_conf = {
    'parties': {
        'alice': {
            'address': 'alice:8080',
        },
        'bob': {
            'address': 'bob:8080',
        },
    },
}

### alice 和 bob 初始化

<span style="color: #BA4081">注意：下面初始化语句是分别只在对应的 alice 或 bob 节点执行的。除了初始化语句，后面所有代码，会被同时下发到 alice 和 bob 节点执行。</span>

In [2]:
sf.shutdown()
sf.init(
    address='127.0.0.1:6379',
    cluster_config={**network_conf, 'self_party': 'alice'},
    log_to_driver=True,
)

Alice (You)'s Output:

2023-10-10 05:18:22,228	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 172.21.0.2:6379...
2023-10-10 05:18:22,253	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at http://172.21.0.2:8265 
2023-10-10 05:18:22 INFO api.py:147 [alice] --  Started rayfed with {'CLUSTER_ADDRESSES': {'alice': 'alice:8080', 'bob': 'bob:8080'}, 'CURRENT_PARTY_NAME': 'alice', 'TLS_CONFIG': {}}
2023-10-10 05:18:22 INFO cleanup.py:58 [alice] --  Start check sending thread.
2023-10-10 05:18:22 INFO cleanup.py:67 [alice] --  Start check sending monitor thread.
2023-10-10 05:18:23 INFO barriers.py:235 [alice] --  Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=626) 2023-10-10 05:18:23 INFO grpc_proxy.py:306 [alice] --  ReceiveProxy binding port 8080, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"methodConfig": [{"name":

In [2]:
sf.shutdown()
sf.init(
    address='127.0.0.1:6379',
    cluster_config={**network_conf, 'self_party': 'bob'},
    log_to_driver=True,
)

Bob's Output:

2023-10-10 05:18:22,223	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 172.21.0.3:6379...
2023-10-10 05:18:22,249	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at http://172.21.0.3:8265 
2023-10-10 05:18:22 INFO api.py:147 [bob] --  Started rayfed with {'CLUSTER_ADDRESSES': {'alice': 'alice:8080', 'bob': 'bob:8080'}, 'CURRENT_PARTY_NAME': 'bob', 'TLS_CONFIG': {}}
2023-10-10 05:18:22 INFO cleanup.py:58 [bob] --  Start check sending thread.
2023-10-10 05:18:22 INFO cleanup.py:67 [bob] --  Start check sending monitor thread.
2023-10-10 05:18:23 INFO barriers.py:235 [bob] --  Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=598) 2023-10-10 05:18:23 INFO grpc_proxy.py:306 [bob] --  ReceiveProxy binding port 8080, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"methodConfig": [{"name": [{"service"

## 初始化 SPU

1. alice 的 address 请填写可以被 bob 访通的地址，并且选择一个**未被占用的端口** ，注意不要和 Ray 端口冲突。
2. alice 的 listen_addr 可以和 alice address 里的端口一样。
3. bob 的 address 请填写可以被 alice 访通的地址，并且选择一个**未被占用的端口** ，注意不要和 Ray 端口冲突。
4. bob 的 listen_addr 可以和 bob address 里的端口一样。

In [3]:
alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu_conf = {
    "nodes": [
        {
            "party": "alice",
            "address": "alice:8081",
            "listen_addr": "alice:8081"
        },
        {
            "party": "bob",
            "address": "bob:8081",
            "listen_addr": "alice:8081"
        },
    ],
    "runtime_config": {
        "protocol": spu.spu_pb2.SEMI2K,
        "field": spu.spu_pb2.FM128,
        "sigmoid_mode": spu.spu_pb2.RuntimeConfig.SIGMOID_REAL,
    },
}
spu = sf.SPU(cluster_def=spu_conf)

## 隐私求交

<span style="color: rgb(50, 50, 50)">提供 </span>`psi_csv`<span style="color: rgb(50, 50, 50)"> 函数， </span>`psi_csv`<span style="color: rgb(50, 50, 50)"> 将 csv 文件作为输入，并在求交后生成 csv 文件。默认协议为 </span>[**KKRT**](https://eprint.iacr.org/2016/799.pdf)<span style="color: rgb(50, 50, 50)">。</span>

In [4]:
import os

current_dir = os.getcwd()

input_path = {alice: f'{current_dir}/iris_alice.csv', bob: f'{current_dir}/iris_bob.csv'}
output_path = {alice: f'{current_dir}/iris_alice_psi.csv', bob: f'{current_dir}/iris_alice_psi.csv'}
spu.psi_csv('uid', input_path, output_path, 'alice')

Alice (You)'s Output:

2023-10-10 05:18:26,135	WARNING worker.py:1851 -- Traceback (most recent call last):
  File "python/ray/_raylet.pyx", line 823, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 875, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 830, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 834, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 780, in ray._raylet.execute_task.function_executor
  File "/usr/local/lib/python3.8/site-packages/ray/_private/function_manager.py", line 674, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/usr/local/lib/python3.8/site-packages/ray/util/tracing/tracing_helper.py", line 466, in _resume_span
    return method(self, *_args, **_kwargs)
  File "/usr/local/lib/python3.8/site-packages/secretflow/device/device/spu.py", line 370, in __init__
    self.link = spu_link.create_brpc(desc, rank)
RuntimeError: what: 
	[external/yacl/yacl/link/transport/brpc_link.cc:13

(SPURuntime pid=716) 2023-10-10 05:18:26.119 [error] [server.cpp:BRPC:1023] Fail to listen 172.21.0.3:8081
(SPURuntime pid=716) 2023-10-10 05:18:26.132 [warning] [channel.h:~Channel:151] Channel destructor is called before WaitLinkTaskFinish, try stop send thread


SystemExit: 15

Bob's Output:

2023-10-10 05:18:26,340	WARNING worker.py:1851 -- Traceback (most recent call last):
  File "python/ray/_raylet.pyx", line 823, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 875, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 830, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 834, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 780, in ray._raylet.execute_task.function_executor
  File "/usr/local/lib/python3.8/site-packages/ray/_private/function_manager.py", line 674, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/usr/local/lib/python3.8/site-packages/ray/util/tracing/tracing_helper.py", line 466, in _resume_span
    return method(self, *_args, **_kwargs)
  File "/usr/local/lib/python3.8/site-packages/secretflow/device/device/spu.py", line 370, in __init__
    self.link = spu_link.create_brpc(desc, rank)
RuntimeError: what: 
	[external/yacl/yacl/link/transport/brpc_link.cc:13

(SPURuntime pid=683) 2023-10-10 05:18:26.321 [error] [server.cpp:BRPC:1023] Fail to listen 172.21.0.2:8081
(SPURuntime pid=683) 2023-10-10 05:18:26.337 [warning] [channel.h:~Channel:151] Channel destructor is called before WaitLinkTaskFinish, try stop send thread


SystemExit: 15